# Access to Fors2 Spectra and compare with filters and image

Spectra in observation frame

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2024/02/10
- update : 2024/02/15

The purpose is to recalibrate redshift by emission line

The first goal is to find in which frame (rest or observation) the Fors2 spectra are.
This guess is tested against the tabulated resdhift.

In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
import re
from astropy.io import fits
from astropy.table import Table
import matplotlib as mpl
import matplotlib.pyplot as plt
#%matplotlib inline
%matplotlib ipympl
import matplotlib.colors as colors
import matplotlib.cm as cmx
import collections
from collections import OrderedDict
import re
import matplotlib.gridspec as gridspec
from sklearn.gaussian_process import GaussianProcessRegressor, kernels
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
from IPython.core.display import Image, display
from IPython import display

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
from sedpy import observate
# get magnitude from a spectrum:

In [ ]:
from sedpy import observate
from fors2pcigale.filters import FilterInfo

In [ ]:
from fors2pcigale.fors2starlightio import Fors2DataAcess

In [ ]:
Lyman_lines = [1220., 1030. ,973.,950., 938., 930.]
Balmer_lines = [6562.791,4861.351,4340.4721,4101.740,3970.072,3889.0641,3835.3971]
Paschen_lines = [8750., 12820., 10938.0,10050., 9546.2, 9229.7,9015.3, 8862.89,8750.46,8665.02]
Brackett_lines = [40522.79, 26258.71, 21661.178, 19440., 18179.21]
Pfund_lines = [ 74599.0, 46537.8, 37405.76 , 32969.8, 30400.]
all_Hydrogen_lines = [ Lyman_lines, Balmer_lines, Paschen_lines, Brackett_lines, Pfund_lines]
Color_lines = ["purple", "blue", "green", "red","grey"]
Balmer_thres = 3645.6
Lyman_thres = 911.267
Paschen_thres = 8200.
Brackett_thres = 14580.
Pfund_lines = 22800.
all_Hydrogen_thres = [Lyman_thres , Balmer_thres, Paschen_thres, Brackett_thres, Pfund_lines]


In [ ]:
D4000_red = [4050.,4250] 
D4000_blue = [3750.,3950.]
W_BALMER = [Balmer_thres, Balmer_lines[0]]
W_LYMAN = [Lyman_thres, Lyman_lines[0]]

In [ ]:
def plot_hydrogen_lines(ax):
    nth = len(all_Hydrogen_thres)
    for idx,group_lines in enumerate(all_Hydrogen_lines):
        # select only Lyman and Balmer
        if idx<2:
            color = Color_lines[idx]
            for wl_line in group_lines:
                ax.axvline(wl_line,color=color,lw=0.5)
            if idx< nth:
                ax.axvline(all_Hydrogen_thres[idx],color=color,linestyle=":")
    ax.axvspan(W_LYMAN[0],W_LYMAN[1],facecolor='purple', alpha=0.5)
    ax.axvspan(W_BALMER[0],W_BALMER[1],facecolor='green', alpha=0.2)

In [ ]:
def plot_hydrogen_lines_redshift(ax,z):
    nth = len(all_Hydrogen_thres)
    for idx,group_lines in enumerate(all_Hydrogen_lines):
        # select only Lyman and Balmer
        if idx<2:
            color = Color_lines[idx]
            for wl_line in group_lines:
                ax.axvline(wl_line*(1+z),color=color,lw=2)
            if idx< nth:
                ax.axvline(all_Hydrogen_thres[idx]*(1+z),color=color,linestyle="-",lw=2)
    ax.axvspan(W_LYMAN[0]*(1+z),W_LYMAN[1]*(1+z),facecolor='purple', alpha=0.5)
    ax.axvspan(W_BALMER[0]*(1+z),W_BALMER[1]*(1+z),facecolor='green', alpha=0.2)

## Configuration

In [ ]:
# this spectrum must be removed
bad_spectra_list = ["SPEC183"]

## Init

### Filters

In [ ]:
ps = FilterInfo()
ps.dump()

### Init data

In [ ]:
fors2 = Fors2DataAcess()

In [ ]:
#fors2.plot_allspectra()

In [ ]:
fors2_tags = fors2.get_list_of_groupkeys()
len(fors2_tags)

In [ ]:
list_of_fors2_attributes = fors2.get_list_subgroup_keys()
print(list_of_fors2_attributes)

In [ ]:
spec_name_sel = fors2_tags[0]

In [ ]:
spec_sec_img = fors2.get_specimg(spec_name_sel)
spec_sec_fileimg =  fors2.get_specimgfile(spec_name_sel)

In [ ]:
Image(spec_sec_fileimg,width=2000,height=200)

## Loop to plot

In [ ]:
bwr_map = plt.get_cmap('bwr')
reversed_map = bwr_map.reversed() 
cNorm = colors.Normalize(0., vmax=1.)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=bwr_map)

In [ ]:
for specname in fors2_tags:


    if specname in bad_spectra_list:
        print(f">>>> SKIP spectrum {specname} which is in list of bad spectra")
        continue
    
    attr = fors2.getattribdata_fromgroup(specname)

    # acess to the image array
    img = fors2.get_specimg(specname)

    # get the image filename and path
    spec_sec_fileimg = fors2.get_specimgfile(specname)
    
    redshift = attr['redshift']
    lines = attr['lines']
    the_color = scalarMap.to_rgba(redshift, alpha=1)
    the_dict_sed =  fors2.getspectrum_fromgroup(specname)
    wl = the_dict_sed["wl"] 
    fl = the_dict_sed["fl"] 

    filt = observate.Filter("sdss_r0")
    #mag = filt.ab_mag(angstroms, f_lambda_cgs)
    mag = filt.ab_mag(wl, fl)
    
    ymax = fl.max()

    fig = plt.figure(constrained_layout=True,figsize=(12,6))
    plt.imshow(img)
    ax = plt.gca()
    # Hide X and Y axes label marks
    ax.xaxis.set_tick_params(labelbottom=False)
    ax.yaxis.set_tick_params(labelleft=False)
    # Hide X and Y axes tick marks
    ax.set_xticks([])
    ax.set_yticks([])
    plt.show()

    fig = plt.figure(constrained_layout=True,figsize=(12,6))
    #fig = plt.figure(figsize=(12,6))

    gs = gridspec.GridSpec(nrows=2, ncols=1, height_ratios=[3,1],hspace=0)

    #ax0 = fig.add_subplot(gs[0])
    #ax0.imshow(arr[:,:,0])
    
    ax1 = fig.add_subplot(gs[0])

    ax1.plot(wl,fl,'-',color=the_color)
    title = f"{specname}, z={redshift:.2f}, lines = {lines}"
    ax1.set_title(title)
    ax1.axvline(wl.min(),color="k")
    ax1.axvline(wl.max(),color="k")
    ax1.set_xlim(4000.,10000.)
    ax1.set_ylim(0.,ymax)
    ax1.set_xlabel("$\lambda (\\AA)$")
    ax1.set_ylabel("$F_\lambda(\lambda) (A.U)$")
    ax1.grid()
    plot_hydrogen_lines_redshift(ax1,redshift)



    ax2 = fig.add_subplot(gs[1], sharex=ax1)  
    plt.setp(ax2.get_yticklabels(), visible=False)

    # loop on filters
    for index in ps.filters_indexlist:
        the_name = ps.filters_namelist[index]
        the_filt = ps.filters_transmissionlist[index]
        the_norm = ps.filters_transmissionnormlist[index]
        the_wlmean = the_filt.wave_mean
        the_color = ps.filters_colorlist[index]
        the_transmission =the_filt.transmission/the_norm
        ax2.plot(the_filt.wavelength,the_transmission,color=the_color)

        if index%2 ==0:
            ax2.text(the_wlmean, 0.7, the_name,horizontalalignment='center',verticalalignment='center',color=the_color,fontweight="bold")
        else:
            ax2.text(the_wlmean, 0.85, the_name,horizontalalignment='center',verticalalignment='center',color=the_color,fontweight="bold")
    ax2.axvline(wl.min(),color="k")
    ax2.axvline(wl.max(),color="k")
    ax2.grid()
    __=ax2.set_ylim(0.,1.)
  
    axins1 = inset_axes(ax1,
                    width="50%",  # width = 50% of parent_bbox width
                    height="5%",  # height : 5%
                    loc='upper right')
    
    cbar=fig.colorbar(scalarMap , cax=axins1,orientation='horizontal')
    cbar.ax.set_xlabel('redshift')
    #fig.colorbar(im1, cax=axins1, orientation="horizontal", ticks=[1, 2, 3])
    #axins1.xaxis.set_ticks_position("bottom")

    plt.show()
    